<a href="https://colab.research.google.com/github/zshamroukh/Colab/blob/main/api_use_case_peak_to_current.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Contextual Insights in 3 Steps via the Parcl Labs API

#Step 1: Register for the API

You can retrieve Parcl Labs API data with only a couple lines of GraphQL code. For demonstration purposes here we will show you a use case using Python, one of the most popular programming languages for data analysis, but the API can be used in combination with your favorite tech stack. Once you’ve received your key, by signing up [here](https://www.parcllabs.com/contact?utm_source=ParclLabs&utm_medium=Blog&utm_campaign=API-USE-CASE-BLOG) for free, you can store it as a variable in Python along with the API URL:

In [ ]:
#Importing the relevant libraries
import os
import requests
import pandas as pd
import plotly.express as px

#Storing the API credentials as an environment variable
url = "https://api-pilot.parcllabs.com/v1/graphql"
# to set the API key as an environment variable in your CLI issue the following command on MacOS/Linux OS's: export PARCL_LABS_API_PILOT_KEY="INSERT_API_KEY_HERE"
bearer_token = os.environ['PARCL_LABS_API_PILOT_KEY']

# Step 2: Define the GraphQL query

By accessing the price feed through the API, we can explore the price feed fluctuations in the Case Schiller 20 MSAs (metropolitan statistical areas) for 2022. Specifically, in just a few lines of code, we can extract the 2022 peak and current price feed values, and calculate the percentage change.

First we name our query, in this case PEAK_TO_CURRENT_2022, and then we can query all MSAs in the MSA table (see the Parcl Labs API docs for a comprehensive list of geographies and objects available in the API). In the pilot MSA only the Case Schiller 20 MSAs are available.



After defining the level of geography, the columns we output from the MSA table are MSA_NAME and PARCL_ID (our unique identifier of different levels of geography). Within the MSA geographies is a nested table, parcl_price_feed, containing the price feed for each day. In the example below we pull in the most recent price feed, in addition to the max price that occurred in 2022. This is what the query would look like:

In [ ]:
#Storing the query to get the current and 2022 peak prices for the CS 20 MSAs
query = """
query PEAK_TO_CURRENT_2022 {
  MSA {
    MSA_NAME
    PARCL_ID
    parcl_price_feed(order_by: {DATE: desc}, limit: 1) {
      PARCL_PRICE_FEED
    }
    parcl_price_feed_aggregate(where: {DATE: {_gt: "01/01/2022"}}) {
       aggregate {
       	max {
           PARCL_PRICE_FEED
         }
       }
     }
  }
}
"""

#Step 3: Call the API and visualize the results

Once you have plugged in your query, the URL and API key you can call the API with Python:

In [ ]:
# Call the API using our query and credentials
response = requests.post(
   url=url,
   json={
     "query": query
   },
   headers={
     "Authorization": f"Bearer {bearer_token}",
     'content-type': "application/json"
   }
)
 
# Parse the JSON response to native Python object
out = response.json()
data = out['data']['MSA']

The next step is to flatten the JSON response into a dataframe and calculate the percent difference between the peak and current prices:

In [ ]:
#Pull the data into a dataframe and calculate the delta 
PEAK_TO_CURRENT_2022_df = pd.json_normalize(
    data,
    'parcl_price_feed',
    ['MSA_NAME', 'PARCL_ID', ['parcl_price_feed_aggregate', 'aggregate', 'max', 'PARCL_PRICE_FEED']]
)

PEAK_TO_CURRENT_2022_df.rename(columns={'PARCL_PRICE_FEED': 'Current PLPF', 'parcl_price_feed_aggregate.aggregate.max.PARCL_PRICE_FEED': '2022 Peak PLPF'}, inplace=True)
PEAK_TO_CURRENT_2022_df['Percent Delta'] = ((PEAK_TO_CURRENT_2022_df['Current PLPF'] - PEAK_TO_CURRENT_2022_df['2022 Peak PLPF']) / PEAK_TO_CURRENT_2022_df['2022 Peak PLPF'] * 100)
PEAK_TO_CURRENT_2022_df[['Percent Delta', '2022 Peak PLPF', 'Current PLPF']] = PEAK_TO_CURRENT_2022_df[['Percent Delta', '2022 Peak PLPF', 'Current PLPF']].astype(float).round(2)
PEAK_TO_CURRENT_2022_df = PEAK_TO_CURRENT_2022_df.sort_values(by=['Percent Delta'], ascending=True)

print(PEAK_TO_CURRENT_2022_df)

    Current PLPF                                      MSA_NAME PARCL_ID  \
14        144.25               Dallas-Fort Worth-Arlington, TX  2899734   
0         481.73            San Diego-Chula Vista-Carlsbad, CA  2900332   
15        367.34                   Seattle-Tacoma-Bellevue, WA  2900353   
19        679.47            San Francisco-Oakland-Berkeley, CA  2900336   
4         337.27                Boston-Cambridge-Newton, MA-NH  2899625   
7         232.24              Las Vegas-Henderson-Paradise, NV  2900049   
16        183.13            Chicago-Naperville-Elgin, IL-IN-WI  2899845   
12        292.37                    Denver-Aurora-Lakewood, CO  2899750   
17        292.56       Miami-Fort Lauderdale-Pompano Beach, FL  2900128   
11        211.44           Tampa-St. Petersburg-Clearwater, FL  2900417   
10        336.84         New York-Newark-Jersey City, NY-NJ-PA  2900187   
3         249.01                     Phoenix-Mesa-Chandler, AZ  2900245   
8         288.02         

And finally visualize it with your choice of visualization libraries, here we use [Plotly](https://plotly.com/):

In [ ]:
#Plot the percent delta for the MSAs
PEAK_CURRENT_fig = px.bar(
    PEAK_TO_CURRENT_2022_df,
    x='MSA_NAME',
    y='Percent Delta',
    title='2022 Price Feed Peak Compared to Current Values (12/09/22)',
    width=900,
    height=500,
    color='Percent Delta',
    color_continuous_scale=px.colors.sequential.RdBu,
    range_color=[-20,0],
    labels={"MSA_NAME": "MSA"}
)

PEAK_CURRENT_fig.show()